In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F
from google.colab import drive
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn import metrics

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path_prefix = '/content/drive/Shared drives/CS 230 Project (EEG)/'

data = np.load(path_prefix + 's44_psd_f_diff.npy')
# data = np.load(path_prefix + 'combined_subjects_psd_f_diff.npy')
# data = np.load(path_prefix + 's44_psd_cluster_f.npy')
#data = np.load(path_prefix + 'combined_subjects_psd_cluster_b.npy')

#data = np.load(path_prefix + 's44_psd_data_multitaper.npy')
#data = np.load(path_prefix + 's44_psd_data_welch_nonaveraged.npy')

#data1 = np.load(path_prefix + 'combined_subjects_psd_1.npy')
#data2 = np.load(path_prefix + 'combined_subjects_psd_2.npy')
#data3 = np.load(path_prefix + 'combined_subjects_psd_3.npy')
#data4 = np.load(path_prefix + 'combined_subjects_psd_4.npy')
#data5 = np.load(path_prefix + 'combined_subjects_psd_5.npy')

labels = np.load(path_prefix + 's44_labels.npy')
#labels = np.load(path_prefix + 's44_newlabels_3_33_66.npy')
# labels = np.load(path_prefix + 'combined_subjects_labels.npy')

In [20]:
print("Data Shape: " + str(np.shape(data)))
print("Labels Shape: " + str(np.shape(labels)))

Data Shape: (2309, 3, 5)
Labels Shape: (2309, 1)


In [0]:
data = np.reshape(data, (np.shape(data)[0], -1))

dataset_indices = np.arange(data.shape[0])

# set seed to obtain same results across trials
np.random.seed(0)

# shuffle data indices 
np.random.shuffle(dataset_indices)
train_indices = dataset_indices[ : 6 * data.shape[0] // 10]
valid_indices = dataset_indices[ 6 * data.shape[0] // 10 : 8 * data.shape[0] // 10]
test_indices = dataset_indices[ 8 * data.shape[0] // 10 :]

x_train = data[train_indices].astype(np.float32)
y_train = labels[train_indices]
x_valid = data[valid_indices].astype(np.float32)
y_valid = labels[valid_indices]
x_test  = data[test_indices].astype(np.float32)
y_test  = labels[test_indices]

y_train = y_train.squeeze()
y_valid = y_valid.squeeze()
y_test = y_test.squeeze()

In [0]:
# Convert Dataset to Tensors
x_train, y_train, x_valid, y_valid, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test)
)
# Training TensorDataset
train_ds = TensorDataset(x_train, y_train)

# Validation TensorDataset
valid_ds = TensorDataset(x_valid, y_valid)

# Testing TensorDataset
test_ds = TensorDataset(x_test, y_test)

In [8]:
# Start Device
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

print(torch.cuda.is_available())

True


In [0]:
def get_data(train_ds, valid_ds, test_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
        DataLoader(test_ds, batch_size=bs * 2)
    )

def preprocess(x, y):
    return x.to(dev), y.to(dev)

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b)) 

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [0]:
def fit(model, trainloader, validloader, opt, loss_function, epochs):
  for epoch in range(epochs):
    train_loss, valid_loss = [], []
    model.train()

    for data, target in trainloader:
      data = data.view(data.shape[0], -1)
      opt.zero_grad()
      output = model(data) #Forward prop
      #if len(train_loss) == 0:
      #  print(output)

      #print(output.shape)
      # print(target.shape)
      # target = target.unsqueeze(1).long()
      # print(target.shape)

      loss = loss_function(output, target.unsqueeze(1).type_as(output)) #Calculate loss
      #loss = loss_function(output, target.long())

      loss.backward() #Backward prop
      opt.step() #Weight optimization

      train_loss.append(loss.item())
    
    model.eval()
    for data, target in validloader:
      data = data.view(data.shape[0], -1)
      output = model(data)
      loss = loss_function(output, target.unsqueeze(1).type_as(output))
      #loss = loss_function(output, target.long())
      valid_loss.append(loss.item())

    #acc1, prec1, rec1, f11 = test(model, trainloader)
    #acc2, prec2, rec2, f12 = test(model, validloader)
    #print("Train Acc: " + str(acc1) + ", Valid Acc: " + str(acc2))
    
    #print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

In [0]:
bs = 64
lr = 0.0001
epochs = 10

#loss_func = nn.CrossEntropyLoss()
#loss_func = F.cross_entropy
loss_func = nn.BCEWithLogitsLoss()

train_dl, valid_dl, test_dl = get_data(train_ds, valid_ds, test_ds, bs)

# Training Data Loader
train_dl = WrappedDataLoader(train_dl, preprocess)

# Validation Data Loader
valid_dl = WrappedDataLoader(valid_dl, preprocess)

# Test Data Loader
test_dl = WrappedDataLoader(test_dl, preprocess)

input_size = 150

# model = nn.Sequential(
#     nn.Linear(input_size, layer2),
#     nn.BatchNorm1d(layer2),
#     nn.Tanh(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer2, 1)
# )

model = nn.Sequential(
    nn.Linear(input_size, layer2),
    nn.BatchNorm1d(layer2),
    nn.ReLU(),
    nn.Linear(layer2, layer3),
    nn.Dropout(p=0.5),
    nn.BatchNorm1d(layer3),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(layer3, 1)

    #Lambda(lambda x: x.view(x.size(0), -1))
)

# model = nn.Sequential(
#     nn.Linear(input_size, layer2),
#     nn.BatchNorm1d(layer2),
#     nn.ReLU(),
#     nn.Linear(layer2, layer3),
#     nn.BatchNorm1d(layer3),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer3, layer4),
#     nn.BatchNorm1d(layer4),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer4, 1)

#     #Lambda(lambda x: x.view(x.size(0), -1))
# )
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [0]:
10**(np.random.uniform(low=-6, high=-1))

3.4293590459565007e-06

In [0]:
bs = 64
train_dl, valid_dl, test_dl = get_data(train_ds, valid_ds, test_ds, bs)

# Training Data Loader
train_dl = WrappedDataLoader(train_dl, preprocess)

# Validation Data Loader
valid_dl = WrappedDataLoader(valid_dl, preprocess)

# Test Data Loader
test_dl = WrappedDataLoader(test_dl, preprocess)

In [24]:
#loss_func = nn.BCEWithLogitsLoss()

loss_func = nn.BCELoss()
#loss_func = nn.CrossEntropyLoss()

iterations = 500
models = []
good_models = {}
for i in range(iterations):
  input_size = 15
  output_size = 1
  layer2 = int(np.random.uniform(low=input_size, high=2000))
  #layer2 = 1500
  layer3 = int(np.random.uniform(low=10, high=layer2))
  #layer3 = 400
  layer4 = int(np.random.uniform(low=8, high = layer3))
  #layer4 = 20
  dropout = np.random.uniform(low=0, high=0.9)
  #dropout = 0.5
  momentum = np.random.uniform(low=0.5, high=0.99)
  lr = 10**(np.random.uniform(low=-6, high=-1))
  #lr = 1e-5
  epochs = int(np.random.uniform(low=5, high=30))
  #epochs = 20

  print("Layer2: " + str(layer2) + ", Layer3: " + str(layer3) + ", Drop: " + str(dropout) + ", Momentum: " + str(momentum) + ", LR: " + str(lr) + ", Epochs: " + str(epochs)) 
  model = nn.Sequential(
      nn.Linear(input_size, layer2),
      nn.BatchNorm1d(layer2),
      nn.ReLU(),
      nn.Linear(layer2, layer3),
      nn.BatchNorm1d(layer3),
      nn.ReLU(),
      nn.Dropout(p=dropout),
      nn.Linear(layer3, output_size),
      nn.Sigmoid()
  )

  # model = nn.Sequential(
  #     nn.Linear(input_size, layer2),
  #     nn.BatchNorm1d(layer2),
  #     nn.ReLU(),
  #     nn.Linear(layer2, layer3),
  #     nn.BatchNorm1d(layer3),
  #     nn.ReLU(),
  #     nn.Linear(layer3, layer4),
  #     nn.BatchNorm1d(layer4),
  #     nn.ReLU(),
  #     nn.Dropout(p=dropout),
  #     nn.Linear(layer4, output_size),
  #     nn.Sigmoid()
  # )

  # model = nn.Sequential(
  #     nn.Linear(input_size, layer2),
  #     nn.BatchNorm1d(layer2),
  #     nn.ReLU(),
  #     nn.Dropout(p=dropout),
  #     nn.Linear(layer2, 1),
  #     nn.Sigmoid()
  # )

  #opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
  opt = optim.Adam(model.parameters(), lr=lr)
  model.to(dev)
  fit(model, train_dl, valid_dl, opt, loss_func, epochs)
  models.append(model)
  acc, prec, rec, f1 = test(model, test_dl)
  print(str(i) + " - Test:  Acc: " + str(acc) + ", Prec: "  + str(prec) + ", Rec: " + str(rec) + ", F1: " + str(f1))
  if acc > 0.67:
    #print("Good model")
    good_models[i] = {'layer2':layer2, 'layer3':layer3, 'layer4':layer4, 'dropout':dropout, 'momentum':momentum, 'lr':lr, 'epochs':epochs, 'acc':acc, 'prec':prec, 'rec':rec, 'f1':f1}

Layer2: 1564, Layer3: 1017, Drop: 0.7740239124736775, Momentum: 0.8028558213046733, LR: 0.05666312231406209, Epochs: 20
0 - Test:  Acc: 0.6991341991341992, Prec: 0.7272727272727273, Rec: 0.62882096069869, F1: 0.674473067915691
Layer2: 1631, Layer3: 1610, Drop: 0.4422420632311081, Momentum: 0.7883515296766921, LR: 0.0007349092909939168, Epochs: 6
1 - Test:  Acc: 0.6796536796536796, Prec: 0.7098445595854922, Rec: 0.5982532751091703, F1: 0.6492890995260663
Layer2: 779, Layer3: 334, Drop: 0.03717571220918227, Momentum: 0.5023262771191965, LR: 6.081705779809254e-05, Epochs: 11
2 - Test:  Acc: 0.696969696969697, Prec: 0.7236180904522613, Rec: 0.62882096069869, F1: 0.6728971962616822
Layer2: 1117, Layer3: 1032, Drop: 0.3917395434128778, Momentum: 0.8271464046134702, LR: 2.636558937350567e-06, Epochs: 20
3 - Test:  Acc: 0.6904761904761905, Prec: 0.7087378640776699, Rec: 0.6375545851528385, F1: 0.671264367816092
Layer2: 747, Layer3: 574, Drop: 0.11114570281848743, Momentum: 0.5524508697980227, 

In [25]:
for good in good_models:
  if good_models[good]['acc'] > 0.716:
    print(str(good) + ": Acc = " + str(good_models[good]['acc']) + ", F1: " + str(good_models[good]['f1']))
    print(good_models[good])
    print("")

74: Acc = 0.7164502164502164, F1: 0.7002288329519452
{'layer2': 74, 'layer3': 34, 'layer4': 11, 'dropout': 0.17228298636622236, 'momentum': 0.8868737697207121, 'lr': 0.0002295886012470785, 'epochs': 25, 'acc': 0.7164502164502164, 'prec': 0.7355769230769231, 'rec': 0.6681222707423581, 'f1': 0.7002288329519452}

124: Acc = 0.7164502164502164, F1: 0.7069351230425056
{'layer2': 1523, 'layer3': 495, 'layer4': 359, 'dropout': 0.863692542303969, 'momentum': 0.783828458179742, 'lr': 0.0014886868062570459, 'epochs': 18, 'acc': 0.7164502164502164, 'prec': 0.7247706422018348, 'rec': 0.6899563318777293, 'f1': 0.7069351230425056}



In [26]:
model = models[74]
print('Test Accuracy:')
acc, prec, rec, f1 = test(model, test_dl)
print("   Acc: " + str(acc) + ", Prec: "  + str(prec) + ", Rec: " + str(rec) + ", F1: " + str(f1))

Test Accuracy:
   Acc: 0.7164502164502164, Prec: 0.7355769230769231, Rec: 0.6681222707423581, F1: 0.7002288329519452


In [27]:
PATH = './s44_psd_f_diff_1.pth'
print('Save state of network...')
answer = None
while answer not in ("Y", "N"):
    answer = input("Enter Y or N: ")
    if answer == "Y": # Save the current state of the network.
       
        torch.save(model.state_dict(), PATH)
        print('State of the network was saved.')
    elif answer == "N": # Move on without saving.
        print('State of the network was NOT saved.')
        continue
    else:
        print("Please enter yes or no.")

Save state of network...
Enter Y or N: Y
State of the network was saved.


In [0]:
def test_CEL(model, test_dl):
    
    all_ground = []
    all_pred = []
    
    for xb, yb in test_dl:
        raw_predict = model.forward(xb).argmax(1)
        predictions = raw_predict.squeeze()

        all_ground.append(yb.view(-1).cpu().numpy())
        all_pred.append(predictions.view(-1).cpu().numpy())

    all_ground = np.hstack(all_ground)
    all_pred = np.hstack(all_pred)

    conf_matrix = metrics.confusion_matrix(all_ground, all_pred)
    print(conf_matrix)
    print(metrics.classification_report(all_ground, all_pred))

    return 5, 5, 5, 5


In [0]:
def test(model, test_dl):
    nums = 0
    correct_predictions = 0

    #Precision - how many positive predictions were correct 
    pos_pred = 0
    pos_true = 0

    #Recall - how many real positive values were detected
    pos_all = 0
    
    for xb, yb in test_dl:
        raw_predict = model.forward(xb)
        #print(raw_predict)
        predictions = torch.round(raw_predict).squeeze()
        #print(predictions)
        #print(predictions.size())
        predictions[predictions!=0] = 1
        #print("Predictions: " + str(predictions.squeeze()))

        #Calculate precision
        pred_pos = (predictions == 1)
        pred_pos = torch.sum(pred_pos)
        true_pos = (predictions == 1) * (yb == 1)
        #print(predictions.size())
        #print(yb.size())
        #print(true_pos.size())
        #print("X: " + str(true_pos))
        true_pos = torch.sum(true_pos)
        pos_pred += pred_pos.cpu().numpy()
        pos_true += true_pos.cpu().numpy()

        #print("True Positives: " + str(true_pos.cpu().numpy()))
        #print("Positive Predictions: " + str(pred_pos.cpu().numpy()))

        #Calculate recall
        all_pos = (yb == 1)
        all_pos = torch.sum(all_pos)
        pos_all += all_pos.cpu().numpy()

        #print("Actual Positives: " + str(all_pos.cpu().numpy()))

        compare = (predictions == yb)
        correct = torch.sum(compare)
        correct_predictions = correct_predictions + correct
        nums = nums + yb.shape[0]
        #print(correct_predictions)
    test_accuracy = correct_predictions.cpu().numpy() / nums
    precision = pos_true/pos_pred
    recall = pos_true/pos_all
    f1 = 2*precision*recall/(precision + recall)
    #print("Accuracy: " + str(test_accuracy))
    #print("Precision: " + str(precision))
    #print("Recall: " + str(recall))
    #print("F1: " + str(f1))

    return test_accuracy, precision, recall, f1

In [0]:
_, _, _, _ = test(model, test_dl)